# project_week3 - 1026 (google) - 2

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import statistics as stat
from googletrans import Translator
import re

In [5]:
## asking the user an input for the ingredients he/she has left

ingredients=input("Insira as sobras (max. 3): ")
ingredientslst = ingredients.split(",")

                    ############################################################

## web scrapping continente website

def get_ingredients(ingredient):
    
    url = f"https://www.continente.pt/pesquisa/?q={ingredient}+continente&start=0&srule=Continente&pmin=0.01"
    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    
    products_dict = {}
    index = 0
   
    products = soup.find_all("div", attrs = {"ct-inner-tile-wrap col-inner-tile-wrap row no-gutters justify-content-center align-content-start"})
    
    for product in products:
        product_name = product.find_all('a', attrs = {"class":"pwc-tile--description col-tile--description"})[0].get_text()
        product_brand = product.find_all('p', attrs = {"class":"pwc-tile--brand col-tile--brand"})[0].get_text()
        product_cost = ",".join(product.find_all('span', attrs = {"class":"sales pwc-tile--price-primary col-tile--price-primary"})[0].get_text().split())
        product_quantity = product.find_all("p", attrs = {"class":"pwc-tile--quantity col-tile--quantity"})[0].get_text()
        products_dict[index] = {"Name":product_name,
                                "Brand":product_brand,
                                "Cost [€]":product_cost,
                                "Quantity":product_quantity}
        index += 1
    
    return pd.DataFrame.from_dict(products_dict, orient="index")[:1] ## returning the first result of the search
    
    
## function to convert the cost from "1€" to "1" using ReGex

def extract_first_number(intro_string):
    match = re.search(r'[€$]?\d+\,\d{2}', intro_string)
    if match:
        price_str = match.group()
        price = float(price_str.replace('€', '').replace('$', '').replace(',','.'))
        return price
    else:
        return None

## actually converting it and concating the results if input is more than 1 ingredient (function takes 1 argument)
## and returns the first result

continente_ingredients = []
for item in ingredientslst:
    continente_ingredient = get_ingredients(item)
    continente_ingredient["Cost [€]"] = continente_ingredient["Cost [€]"].apply(extract_first_number)
    continente_ingredients.append(continente_ingredient)

## displaying the list of products
## CHANGE: Set in a variable in order to run the extract number function
df_ingredients = pd.concat(continente_ingredients)
display(df_ingredients)

Insira as sobras (max. 3): leite, ovos, farinha


,Name,Brand,Cost [€],Quantity
0,Leite UHT Meio Gordo,Continente,0.78,emb. 1 lt
0,Ovos de Solo Classe L,Continente,2.34,emb. 12 un
0,Farinha de Trigo T55,Continente,0.75,emb. 1 kg


In [6]:
# CHANGE: Changed the google trans part with deepl
# function that returns the recipe's name according to the leftovers, using and api
# list to add the translated ingredients. It needs to be outside the function, in order to be referred later when looping the df_recipes["ID"].

ingredients_translated_list = []

def get_ing_recipe(ingredients):
    
    ##translating the input from pt to en, because the recipes api is built inenglish
    translator = Translator()
    ingredients_translated = translator.translate(ingredients, dest='en').text 
        
    # add the translated word to a list
    for word in ingredients_translated.split(","):
        ingredients_translated_list.append(word)
    
    # API to get the recipes
    url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/recipes/findByIngredients"
    querystring = {"ingredients":ingredients_translated,
                   "number":"5",
                   "ignorePantry":"true",
                   "ranking":"1"}
    headers = {"X-RapidAPI-Key": "2aab6360ccmsh49dd1ea35854c8bp1fe6d7jsn8d4835fc6865",
               "X-RapidAPI-Host": "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com"}
    response = requests.get(url, headers=headers, params=querystring)
    r = response.json()
    
    recipes = {}
    recipe_number = 1

    for i in range(0, len(r)):
        recipe = r[i]['title']
        id_name = r[i]["id"]
        if r[i]['missedIngredients']:
            missing_ingredient = r[i]['missedIngredients'][0]['name']
        else:
            missing_ingredient = "n/a"
        recipes[recipe_number] = {"ID":id_name,
                                   "Recipe":recipe,
                                  "Missing Igredients":missing_ingredient}
    
        recipe_number += 1
            
    return pd.DataFrame.from_dict(recipes, orient="index")[:6] ## first 5 results

df_recipes = get_ing_recipe(ingredients)
#display(df_recipes)

recipe_instructions = []

for i in df_recipes["ID"]:
    url = f"https://api.spoonacular.com/recipes/{i}/information"
    if len(ingredients_translated_list)<2:
        params = {"apiKey": "7af8ef958bd4489690a266f8df4ede95", "ingredients": f"{ingredients_translated_list[0]}"}
    elif len(ingredients_translated_list)<3:
        params = {"apiKey": "7af8ef958bd4489690a266f8df4ede95", "ingredients": f"{ingredients_translated_list[0]},{ingredients_translated_list[1]}"}
    elif len(ingredients_translated_list)<4:
        params = {"apiKey": "7af8ef958bd4489690a266f8df4ede95", "ingredients": f"{ingredients_translated_list[0]},{ingredients_translated_list[1]},{ingredients_translated_list[2]}"}
    response = requests.get(url, params=params)
    recipe_details = response.json()
    recipe_text = recipe_details.get("instructions") # Mariko note: deleted the "None", but I am not sure what I have changed by deleting it.
    recipe_instructions.append(recipe_text)

# Add the instruction list to a new colum of the dataset of recipes
df_recipes["Recipe Instruction"] = recipe_instructions

# Filter out the ID column
df_recipes = df_recipes[['Recipe', 'Missing Igredients', "Recipe Instruction"]]

# Set the option to display long column values without truncation
pd.set_option('display.max_colwidth', None)

display(df_recipes)

,Recipe,Missing Igredients,Recipe Instruction
1,Popovers,butter,"Position a rack in the lower third of the oven; preheat to 450 degrees. Grease a standard popover pan. Using a blender, mix the eggs, milk, flour and salt at low speed until just combined, about 20 seconds. Add the melted butter; blend for 10 seconds.Pour about 1/3 cup of the batter into each prepared cup (filling about halfway). Bake for 20 minutes. Lower the oven temperature to 350 degrees. Bake until puffed, golden and crisp, 15 to 20 minutes. Using a small, sharp knife, carefully cut around the sides of each popover to release. Serve immediately."
2,The Biggest Fattest Fluffiest Popover,butter,"InstructionsPlace the eggs in a medium bowl and cover with hot water (as hot as your tap will go). Let them sit for 10 minutes.Preheat the oven to 450F and move the oven rack to a low position (to prepare for huge popovers!). Grease the popover pan thoroughly inside each cup as well as around the top edges.With your whisk, beat together the eggs, milk and salt until they are combined. The yolks should be completely blended in.Add the flour and whisk until the mixture is frothy and all large lumps are gone. Then quickly stir in the melted butter.Divide the batter between the prepared popover cups. They should be about 3/4 of the way full.Bake the popovers for 20 minutes and then reduce the heat to 350F. Bake for another 10 or 15 minutes. The popovers will be very golden brown.My preference is to serve immediately and enjoy them warm!"
3,Make Crêpes,butter,None
4,Basic Buckwheat Crêpes,butter,"In a blender or food processor, whirl eggs, all-purpose and buckwheat flours, and milk until smooth, scraping down sides of container as necessary. The buckwheat flour may settle as batter stands; stir before cooking if necessary. Set a nonstick frying pan 8 inches across bottom over medium-high heat; when hot, brush bottom with butter, then wipe out excess with a paper towel. Lift pan from heat and pour in 1/4 cup batter; immediately tilt pan and swirl batter to coat bottom. Crpe should set at once and form tiny bubbles. Set pan back over heat and cook crpe until lightly browned at the edge and dry-looking on the surface, 1 to 3 minutes. Run a wide spatula under crpe edge to loosen. Turn crpe over and fill immediately."
5,Popovers,▢ butter,"Instructions\n\nPreheat Oven: Preheat oven to 400°F.\n\nPrep Pan: Grease the popover/muffin tray well with the butter. You can either use softened butter to easily grease the popover cups, or melted butter and use a brush to thoroughly brush the cups. You can pour any leftover butter in the bottom of each cup. Place the pan in the oven and preheat it for 2 minutes.\n\nBlend Ingredients: Add all remaining ingredients to a blender and blend until all ingredients are thoroughly mixed. The batter will look similar to a crepe batter.\n\nBake: Pour the batter in the preheated muffin cups ¾ of the way through. You should have enough to make 12 popovers. Transfer the pan to the oven on the middle rack and place a large baking sheet on the lower rack, in case there's any spillage. Bake for 30 minutes. Make sure to check them so they don't over-brown. Gently slip them out of the pan and serve."


In [7]:
# Function to calculate the cost (updated ver.)

def calculate_cost(dataset):
    
    total_cost = round(sum(dataset["Cost [€]"]), 2)
    return f"Making this recipe will save you €{total_cost}!"

calculate_cost(df_ingredients)

'Making this recipe will save you €3.87!'